# Description:
This notebook focuses on creating a prompt iteratively which will generate the 2nd scoring criteria. This scoring criteria will then be used by later assignments to give scores to each of the must have skills, where the scores will vary between 0 and 5. The purpose of breaking this scoring criteria into 2 steps (criteria generation and score generation using criteria) is to use the concept of chain-of-thoughts to enhance the performance of the OpenAI GPT-3.5 model which is relatively cheaper than directly using GPT-4 model

Now install necessary libraries required to execute all functionalities within this notebook.

In [ ]:
!pip install openai
!pip install PyMuPDF
!pip install textract
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 7.7 MB/s eta 0:00

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184487 sha256=9c56fe5f1c6e501ff894089d2ae0f8d320550d748d7e2c4e7b47c043d1c7356a
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


Upload the .env file to the directory `/content/` which contains the "OPENAI_API_KEY"

The provided code snippet accesses sensitive values like the OpenAI API key


In [ ]:
# Export your API Key to environment variable
# Upload the .env file to the directory "/content/"
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()


True

In [ ]:
import openai
import os
# Retrieve the API key from environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")

# Set the API key for OpenAI
openai.api_key = openai_api_key

Upload the json file containing important information about the Job requirements which was generated in Assignment1

In [ ]:
from google.colab import files


# Upload the second file
print("Please upload the second file (requirements_output.json):")
uploaded2 = files.upload()

# Check to ensure a file was uploaded. If not, prompt again.
while len(uploaded2) == 0:
    print("No file uploaded. Please upload the second file (requirements_output.json) again:")
    uploaded2 = files.upload()

# Merge the dictionaries to have all uploaded files in one
uploaded = {**uploaded2}

# Print details of uploaded files
for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))


Please upload the second file (requirements_output.json):


Saving requirements_output (3).json to requirements_output (3).json
User uploaded file "requirements_output (3).json" with length 810 bytes


`Import Statements`: Essential libraries are imported. These include:
1. `openai` for interacting with the OpenAI API.
2. Utilities like `json`, `os`, and `re` for handling data and file operations.
3. Libraries for reading specific file formats (`docx` for Word documents, `textract` for older Word format, and `fitz` for PDFs).
4. `pandas` for reading Excel files.


`read_requirements` Function:

1. Reads job requirements from a JSON file and returns the data. It includes error handling to manage potential reading errors.

In [ ]:
import openai
import json
import os
from collections import OrderedDict
import re
from docx import Document
import textract
import fitz  # PyMuPDF
import pandas as pd
import math


def read_requirements(file_path):
    # Reads the job requirements from a JSON file
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        return data
    except Exception as e:
        print(f"Error reading requirements JSON: {e}")
        return None

The code below initializes two primary components: job requirements and must-have skills. The `read_requirements` function fetches the job requirements from a predefined JSON file.

In [ ]:
job_requirements = read_requirements('/content/requirements_output.json')
must_have_skills = job_requirements["must_have_skills"]

### *Goal*: To generate a criterion that helps calculate scores for each of the must-have skills present inside a resume.

### *Reason*: First we start with a basic prompt asking the assistant to generate a criterion to help calculate scores for each of the must have skills present inside the resume. The initial approach aimed to provide the assistant with a basic idea of what is expected in terms of the scoring criterion. The method was to guide the model in a step-by-step manner to think about how to evaluate a resume based on specific must-have skills.
# Prompt Version 1:

In [ ]:
def criterion_gen(must_have):
    model="gpt-3.5-turbo-16k"
    max_tokens=4000
    prompt_v1 = f'''I have a resume. I want to find a person whose resume has skills in {must_have}. \
    Let's think step by step on how to look into a resume and give score based on each of the skill mentioned in {must_have}. \
    As an example, take the case where the must have skills are "SKILL1, SKILL2, SKILL3, SKILL4, SKILL5". \
    Now generate a criterion to score each of the skill between 0 and 5.'''

    messages = [
            {"role": "system", "content": f"{prompt_v1}"},
        ]
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            temperature=0,
            max_tokens=max_tokens
        )
    generated_texts = [
        choice.message["content"].strip() for choice in response["choices"]
    ]
    return generated_texts[0]



criterion_gen_output = criterion_gen(must_have_skills)
print(f'''[Criterion Generation Request] ''', criterion_gen_output)


[Criterion Generation Request]  To score each skill between 0 and 5, we can use the following criterion:

1. Skill Relevance: Assess the relevance of the skill to the desired skills. Give a score of 5 if the skill is an exact match, 4 if it is closely related, 3 if it is somewhat related, 2 if it is vaguely related, and 1 if it is not related at all.

2. Skill Proficiency: Evaluate the proficiency level of the skill mentioned in the resume. Give a score of 5 if the candidate has extensive experience and deep knowledge in the skill, 4 if they have significant experience and good knowledge, 3 if they have some experience and basic knowledge, 2 if they have limited experience and basic understanding, and 1 if they have no experience or knowledge in the skill.

3. Skill Application: Consider the practical application of the skill in real-world projects or work experience. Give a score of 5 if the candidate has successfully applied the skill in multiple projects, 4 if they have applied it i

**Note:**
The output generated by the above cell is not properly formatted. Also the output keeps on changing on multiple runs of the above cell.

# Output:


```
[Criterion Generation Request]  To score each skill between 0 and 5, we can use the following criterion:

1. Skill Relevance: Assess the relevance of the skill to the desired skills. Give a score of 5 if the skill is an exact match, 4 if it is closely related, 3 if it is somewhat related, 2 if it is vaguely related, and 1 if it is not related at all.

2. Skill Proficiency: Evaluate the proficiency level of the skill mentioned in the resume. Give a score of 5 if the candidate has extensive experience and deep knowledge in the skill, 4 if they have significant experience and good knowledge, 3 if they have some experience and basic knowledge, 2 if they have limited experience and basic understanding, and 1 if they have no experience or knowledge in the skill.

3. Skill Application: Consider the practical application of the skill in real-world projects or work experience. Give a score of 5 if the candidate has successfully applied the skill in multiple projects, 4 if they have applied it in a few projects, 3 if they have some experience applying it, 2 if they have limited experience, and 1 if they have no practical application of the skill.

By using these criteria, we can assign a score between 0 and 5 for each skill mentioned in the resume.
```



###*Goal*: Enhance the specificity of the criterion by providing an example string (`string_match`) that gives a scoring guide for a dummy skill (`SKILL1`).

###*Reason*: Taking inspiration from the above output we pass a `string_match` string containing an example of the scoring criterion for a dummy skill `SKILL1`. The initial output might not be specific enough for the application. By giving a tangible example of the scoring metric, the aim was to align the model's output more closely with the desired format.
# Prompt Version 2:

In [ ]:
string_match = """
skill1:
No projects: 0
1 project or mention: one or two
2 or 3 projects or mentions: three or four
more than or equal to five projects or mentions: five
"""
def criterion_gen(must_have):
    model="gpt-3.5-turbo-16k"
    max_tokens=4000
    prompt_v2 = f'''I have a resume. I want to find a person whose resume has skills in {must_have}. \
    Let's think step by step on how to look into a resume and give score based on each of the skill mentioned in {must_have}. \
    As an example, take the case where the must have skills are "SKILL1, SKILL2, SKILL3, SKILL4, SKILL5". Now score each of the skill between 0 and 5.\
    For example use the criterion given in {string_match}'''

    messages = [
            {"role": "system", "content": f"{prompt_v2}"},
        ]
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            temperature=0,
            max_tokens=max_tokens
        )
    generated_texts = [
        choice.message["content"].strip() for choice in response["choices"]
    ]
    return generated_texts[0]

criterion_gen_output = criterion_gen(must_have_skills)
print(f'''[Criterion Generation Request] ''', criterion_gen_output)

[Criterion Generation Request]  To find a person whose resume has skills in ['TensorFlow', 'Keras', 'PyTorch', 'Computer Vision'], we can follow these steps:

1. Read the resume and extract the relevant information.
2. Assign a score to each skill based on the mentioned criterion.
3. Calculate the overall score for the resume based on the scores of individual skills.
4. Compare the overall scores of different resumes to find the one with the highest score.

Here's an example of how we can score each skill based on the mentioned criterion:

Skill1 (TensorFlow):
- No projects or mentions: 0
- 1 project or mention: 1 or 2
- 2 or 3 projects or mentions: 3 or 4
- More than or equal to five projects or mentions: 5

Skill2 (Keras):
- No projects or mentions: 0
- 1 project or mention: 1 or 2
- 2 or 3 projects or mentions: 3 or 4
- More than or equal to five projects or mentions: 5

Skill3 (PyTorch):
- No projects or mentions: 0
- 1 project or mention: 1 or 2
- 2 or 3 projects or mentions: 3 or

**Note:**
In the above case the criteria does not focus on the output format. We need to instruct the assistant to give the output in such a format so that it can be used by other applications and for that JSON format is the ideal one

# Output:

```
[Criterion Generation Request]  To score each skill mentioned in the resume, we can follow these steps:

1. Read the resume and extract the skills mentioned.
2. For each skill, count the number of projects or mentions related to that skill.
3. Assign a score based on the count of projects or mentions using the given criterion.
4. Repeat the process for each skill mentioned in the resume.
5. Calculate the overall score by summing up the scores of all the skills.

Let's apply this step-by-step process to the example skills ['Keras', 'TensorFlow', 'PyTorch', 'Computer Vision']:

1. Read the resume and extract the skills mentioned.
   - Assume the resume mentions the following skills: ['Keras', 'TensorFlow', 'PyTorch', 'Computer Vision', 'Keras', 'PyTorch']

2. For each skill, count the number of projects or mentions related to that skill.
   - 'Keras': 2 mentions
   - 'TensorFlow': 1 mention
   - 'PyTorch': 2 mentions
   - 'Computer Vision': 1 mention

3. Assign a score based on the count of projects or mentions using the given criterion.
   - 'Keras': 2 mentions -> Score: one or two
   - 'TensorFlow': 1 mention -> Score: one or two
   - 'PyTorch': 2 mentions -> Score: one or two
   - 'Computer Vision': 1 mention -> Score: one or two

4. Repeat the process for each skill mentioned in the resume.
   - 'Keras': 2 mentions -> Score: one or two
   - 'TensorFlow': 1 mention -> Score: one or two
   - 'PyTorch': 2 mentions -> Score: one or two
   - 'Computer Vision': 1 mention -> Score: one or two

5. Calculate the overall score by summing up the scores of all the skills.
   - 'Keras' score: one or two
   - 'TensorFlow' score: one or two
   - 'PyTorch' score: one or two
   - 'Computer Vision' score: one or two

The overall score for this resume would be the sum of the scores for each skill, which would be between 0 and 8.
```



### *Goal*: Adapt the model to produce JSON-like output for scoring justification.

### *Reason*: The output from the above isn't satisfactory so we add a json output in `string_match` to force the final output to look like the JSON output. The desired format for the output seemed to be in JSON, which would make it easier to integrate and analyze in other systems. By showing the model an example of this format, the hope was to nudge it into generating similar structured outputs.

# Prompt Version 3:

In [ ]:
string_match = """
skill1:
No projects: 0
1 project or mention: one or two
2 or 3 projects or mentions: three or four
more than or equal to five projects or mentions: five
{
"SKILL1": {
    "score": 5,
    "justification": "The candidate has significant experience in SKILL1, as mentioned in the resume."
}
}
"""
def criterion_gen(must_have):
    model="gpt-3.5-turbo-16k"
    max_tokens=4000
    prompt_v3 = f'''I have a resume. I want to find a person whose resume has skills in {must_have}. \
    Let's think step by step on how to look into a resume and give score based on each of the skill mentioned in {must_have}. \
    As an example, take the case where the must have skills are "SKILL1, SKILL2, SKILL3, SKILL4, SKILL5". Now score each of the skill between 0 and 5.\
    For example use the criterion given in {string_match}'''

    messages = [
            {"role": "system", "content": f"{prompt_v3}"},
        ]
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            temperature=0,
            max_tokens=max_tokens
        )
    generated_texts = [
        choice.message["content"].strip() for choice in response["choices"]
    ]
    return generated_texts[0]


criterion_gen_output = criterion_gen(must_have_skills)
print(f'''[Criterion Generation Request] ''', criterion_gen_output)

[Criterion Generation Request]  To find a person whose resume has skills in ['TensorFlow', 'Keras', 'PyTorch', 'Computer Vision'], we can follow these steps:

1. Read the resume and extract the relevant information.
2. Look for mentions or projects related to each skill in the resume.
3. Assign a score to each skill based on the number of mentions or projects.
4. Generate a scorecard for each skill, including the score and a justification.
5. Calculate the overall score for the resume based on the scores of individual skills.

Here's an example of how to score each skill:

Skill: TensorFlow
- No projects or mentions: Score 0
- 1 project or mention: Score 1 or 2
- 2 or 3 projects or mentions: Score 3 or 4
- More than or equal to five projects or mentions: Score 5

Skill: Keras
- No projects or mentions: Score 0
- 1 project or mention: Score 1 or 2
- 2 or 3 projects or mentions: Score 3 or 4
- More than or equal to five projects or mentions: Score 5

Skill: PyTorch
- No projects or menti

# Output:


```
[Criterion Generation Request]  To find a person whose resume has skills in ['Keras', 'TensorFlow', 'PyTorch', 'Computer Vision'], we can follow these steps:

1. Read the resume and extract the relevant information.
2. Assign a score to each skill based on the criteria provided.
3. Calculate the overall score for the resume based on the scores of individual skills.
4. Compare the overall score with a threshold to determine if the person has the required skills.

Here's an example of how we can score each skill:

1. Keras:
   - No projects or mentions: 0
   - 1 project or mention: 1
   - 2 or 3 projects or mentions: 3
   - More than or equal to five projects or mentions: 5

2. TensorFlow:
   - No projects or mentions: 0
   - 1 project or mention: 1
   - 2 or 3 projects or mentions: 3
   - More than or equal to five projects or mentions: 5

3. PyTorch:
   - No projects or mentions: 0
   - 1 project or mention: 1
   - 2 or 3 projects or mentions: 3
   - More than or equal to five projects or mentions: 5

4. Computer Vision:
   - No projects or mentions: 0
   - 1 project or mention: 1
   - 2 or 3 projects or mentions: 3
   - More than or equal to five projects or mentions: 5

For each skill, we assign a score based on the criteria mentioned above. The maximum score for each skill is 5.

Once we have the scores for each skill, we can calculate the overall score for the resume by summing up the scores of individual skills.

Finally, we can compare the overall score with a threshold (e.g., 15) to determine if the person has the required skills. If the overall score is equal to or greater than the threshold, we can consider the person as having the required skills.
```



### *Goal*: Increase the number of dummy skills in `string_match` to guide the model further in scoring multiple skills.

### *Reason*: As the model was not generating satisfactory outputs, the strategy was to provide more context by using multiple dummy skills. This would help the model understand the pattern more clearly and reproduce it for the actual must-have skills.
# Prompt Version 4:

In [ ]:
string_match = """
SKILL1 assessment:
No projects: 0
1 project or mention: one or two
2 or 3 projects or mentions: three or four
more than or equal to five projects or mentions: five

SKILL2 assessment:
No projects: 0
1 project or mention: one or two
2 or 3 projects or mentions: three or four
more than or equal to five projects or mentions: five

SKILL3 assessment:
No projects: 0
1 project or mention: one or two
2 or 3 projects or mentions: three or four
more than or equal to five projects or mentions: five

SKILL4 assessment:
No projects: 0
1 project or mention: one or two
2 or 3 projects or mentions: three or four
more than or equal to five projects or mentions: five
{
"SKILL1": {
    "score": 5,
    "justification": "The candidate has significant experience in SKILL1, as mentioned in the resume."
}
"SKILL2": {
    "score": 0,
    "justification": "The candidate has no experience in SKILL2."
}
"SKILL3": {
    "score": 1,
    "justification": "The candidate has some experience in SKILL3"
}
"SKILL4": {
    "score": 3,
    "justification": "The candidate has good experience in SKILL4."
}
}
"""
def criterion_gen(must_have):
    model="gpt-3.5-turbo-16k"
    max_tokens=4000
    prompt_v4 = f'''I have a resume. I want to find a person whose resume has skills in {must_have}. \
    Let's think step by step on how to look into a resume and give score based on each of the skill mentioned in {must_have}. \
    As an example, take the case where the must have skills are "SKILL1, SKILL2, SKILL3, SKILL4, SKILL5". Now score each of the skill between 0 and 5.\
    For example use the criterion given in {string_match}'''

    messages = [
            {"role": "system", "content": f"{prompt_v4}"},
        ]
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            temperature=0,
            max_tokens=max_tokens

        )
    generated_texts = [
        choice.message["content"].strip() for choice in response["choices"]
    ]
    return generated_texts[0]


criterion_gen_output = criterion_gen(must_have_skills)
print(f'''[Criterion Generation Request] ''', criterion_gen_output)

[Criterion Generation Request]  To find a person whose resume has skills in ['TensorFlow', 'Keras', 'PyTorch', 'Computer Vision'], we can follow these steps:

1. Read the resume and extract the relevant information.
2. Check if the resume mentions any of the required skills ('TensorFlow', 'Keras', 'PyTorch', 'Computer Vision').
3. For each skill, assess the candidate's experience level based on the mentioned projects or mentions.
4. Assign a score between 0 and 5 for each skill based on the assessment criteria.
5. Provide a justification for each skill's score based on the candidate's experience mentioned in the resume.

Here's an example of how the assessment can be done:

Resume:
- Skills: ['TensorFlow', 'Keras', 'PyTorch', 'Computer Vision']
- Projects/Mentions:
  - TensorFlow: 3 projects
  - Keras: 1 mention
  - PyTorch: 2 projects
  - Computer Vision: 0 projects

Assessment:

SKILL1 (TensorFlow):
- Projects: 3
- Score: 5
- Justification: The candidate has significant experience in

**Note:**
The output generated by the above cell contains a lot of unnecessary information like as shown below,



```
1. Read the resume and extract the relevant information.
2. Check if the resume mentions any of the required skills ('TensorFlow', 'Keras', 'PyTorch', 'Computer Vision').
3. For each skill, assess the candidate's experience level based on the mentioned projects or mentions.
4. Assign a score between 0 and 5 for each skill based on the assessment criteria.
5. Provide a justification for each skill's score based on the candidate's experience mentioned in the resume.

Here's an example of how the assessment can be done:

Resume:
- Skills: ['TensorFlow', 'Keras', 'PyTorch', 'Computer Vision']
- Projects/Mentions:
  - TensorFlow: 3 projects
  - Keras: 1 mention
  - PyTorch: 2 projects
  - Computer Vision: 0 projects
```



which can harm the final output so we need to focus removing these unwanted information.

# Output:


```
[Criterion Generation Request]  To find a person whose resume has skills in ['TensorFlow', 'Keras', 'PyTorch', 'Computer Vision'], we can follow these steps:

1. Read the resume and extract the relevant information.
2. Check if the resume mentions any of the required skills ('TensorFlow', 'Keras', 'PyTorch', 'Computer Vision').
3. For each skill, assess the candidate's experience level based on the mentioned projects or mentions.
4. Assign a score between 0 and 5 for each skill based on the assessment criteria.
5. Provide a justification for each skill's score based on the candidate's experience mentioned in the resume.

Here's an example of how the assessment can be done:

Resume:
- Skills: ['TensorFlow', 'Keras', 'PyTorch', 'Computer Vision']
- Projects/Mentions:
  - TensorFlow: 3 projects
  - Keras: 1 mention
  - PyTorch: 2 projects
  - Computer Vision: 0 projects

Assessment:

SKILL1 (TensorFlow):
- Projects: 3
- Score: 5
- Justification: The candidate has significant experience in TensorFlow, with 3 mentioned projects.

SKILL2 (Keras):
- Mentions: 1
- Score: 1
- Justification: The candidate has some experience in Keras, with 1 mention.

SKILL3 (PyTorch):
- Projects: 2
- Score: 3
- Justification: The candidate has good experience in PyTorch, with 2 mentioned projects.

SKILL4 (Computer Vision):
- Projects: 0
- Score: 0
- Justification: The candidate has no experience in Computer Vision.

Based on this assessment, the candidate's scores for each skill are as follows:

{
  "TensorFlow": {
    "score": 5,
    "justification": "The candidate has significant experience in TensorFlow, with 3 mentioned projects."
  },
  "Keras": {
    "score": 1,
    "justification": "The candidate has some experience in Keras, with 1 mention."
  },
  "PyTorch": {
    "score": 3,
    "justification": "The candidate has good experience in PyTorch, with 2 mentioned projects."
  },
  "Computer Vision": {
    "score": 0,
    "justification": "The candidate has no experience in Computer Vision."
  }
}

This assessment provides a score and justification for each skill based on the candidate's experience mentioned in the resume.
```



### *Goal*: Double the emphasis on the JSON output format and make the prompt more insistent on replicating the structure of `string_match`.

### *Reason*: The outputs still weren't aligning with expectations. By emphasizing the desired format more strongly like by mentioning `string_match` twice and guiding the model explicitly, the aim was to force the model to generate responses that closely mirror the example.

# Prompt Version 5:

In [ ]:

string_match = """
SKILL1 assessment:
No projects: 0
1 project or mention: one or two
2 or 3 projects or mentions: three or four
more than or equal to five projects or mentions: five

SKILL2 assessment:
No projects: 0
1 project or mention: one or two
2 or 3 projects or mentions: three or four
more than or equal to five projects or mentions: five

SKILL3 assessment:
No projects: 0
1 project or mention: one or two
2 or 3 projects or mentions: three or four
more than or equal to five projects or mentions: five

SKILL4 assessment:
No projects: 0
1 project or mention: one or two
2 or 3 projects or mentions: three or four
more than or equal to five projects or mentions: five

Based on the above criteion the scores can be given in JSON format as shown below,
{
"SKILL1": {
    "score": 5,
    "justification": "The candidate has significant experience in SKILL1, as mentioned in the resume."
}
"SKILL2": {
    "score": 0,
    "justification": "The candidate has no experience in SKILL2."
}
"SKILL3": {
    "score": 1,
    "justification": "The candidate has some experience in SKILL3"
}
"SKILL4": {
    "score": 3,
    "justification": "The candidate has good experience in SKILL4."
}
}
"""
def criterion_gen(must_have):
    model="gpt-3.5-turbo-16k"
    max_tokens=4000

    prompt_v5 = f'''I have a resume. I want to find a person whose resume has skills in {must_have}. \
    Let's think step by step on how to look into a resume and give score based on each of the skill mentioned in {must_have}. \
    As an example, in the case where the must have skills were "SKILL1, SKILL2, SKILL3, SKILL4, SKILL5", here the criterion for \
    judging the resume was {string_match}. So based on the given example of {string_match} give the scoring criterion of {must_have}'''

    messages = [
            {"role": "system", "content": f"{prompt_v5}"},
        ]
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            temperature=0,
            max_tokens=max_tokens
        )
    generated_texts = [
        choice.message["content"].strip() for choice in response["choices"]
    ]
    return generated_texts[0]



criterion_gen_output = criterion_gen(must_have_skills)
print(f'''[Criterion Generation Request] ''', criterion_gen_output)

def save_to_textfile(filename, criterion_text, string_match_text):
    with open(filename, 'w') as file:
        file.write("----Criterion Generated Output----\n")
        file.write(criterion_text)
        file.write("\n\n----String Match----\n")
        file.write(string_match_text)
# Save to text file
save_to_textfile("criterion_and_string_match_output.txt", criterion_gen_output, string_match)

[Criterion Generation Request]  The scoring criterion for the skills ['TensorFlow', 'Keras', 'PyTorch', 'Computer Vision'] can be defined as follows:

TensorFlow assessment:
- No projects: 0
- 1 project or mention: 1
- 2 or 3 projects or mentions: 2
- More than or equal to five projects or mentions: 5

Keras assessment:
- No projects: 0
- 1 project or mention: 1
- 2 or 3 projects or mentions: 2
- More than or equal to five projects or mentions: 5

PyTorch assessment:
- No projects: 0
- 1 project or mention: 1
- 2 or 3 projects or mentions: 2
- More than or equal to five projects or mentions: 5

Computer Vision assessment:
- No projects: 0
- 1 project or mention: 1
- 2 or 3 projects or mentions: 2
- More than or equal to five projects or mentions: 5

Based on the above criteria, the scores for each skill can be given in JSON format as shown below:

{
  "TensorFlow": {
    "score": 0,
    "justification": "The candidate has no experience in TensorFlow."
  },
  "Keras": {
    "score": 2,


# Output:


```
[Criterion Generation Request]  To give the scoring criterion for the skills ['Keras', 'TensorFlow', 'PyTorch', 'Computer Vision'], we can follow a similar approach as mentioned in the example. Here is the scoring criterion:

Keras assessment:
No projects or mentions: 0
1 project or mention: one or two
2 or 3 projects or mentions: three or four
more than or equal to five projects or mentions: five

TensorFlow assessment:
No projects or mentions: 0
1 project or mention: one or two
2 or 3 projects or mentions: three or four
more than or equal to five projects or mentions: five

PyTorch assessment:
No projects or mentions: 0
1 project or mention: one or two
2 or 3 projects or mentions: three or four
more than or equal to five projects or mentions: five

Computer Vision assessment:
No projects or mentions: 0
1 project or mention: one or two
2 or 3 projects or mentions: three or four
more than or equal to five projects or mentions: five

Based on the above criteria, the scores can be given in JSON format as shown below:

{
  "Keras": {
    "score": <score>,
    "justification": "<justification>"
  },
  "TensorFlow": {
    "score": <score>,
    "justification": "<justification>"
  },
  "PyTorch": {
    "score": <score>,
    "justification": "<justification>"
  },
  "Computer Vision": {
    "score": <score>,
    "justification": "<justification>"
  }
}

You can replace `<score>` with the respective score based on the number of projects or mentions in the resume for that skill, and `<justification>` with the justification for the score.

For example:

{
  "Keras": {
    "score": 5,
    "justification": "The candidate has significant experience in Keras, as mentioned in the resume."
  },
  "TensorFlow": {
    "score": 3,
    "justification": "The candidate has good experience in TensorFlow."
  },
  "PyTorch": {
    "score": 2,
    "justification": "The candidate has some experience in PyTorch."
  },
  "Computer Vision": {
    "score": 4,
    "justification": "The candidate has extensive experience in Computer Vision, as mentioned in the resume."
  }
}

You can adjust the scoring criteria and justification based on the specific requirements and expectations for each skill.
```



Iterative prompt development is essential when trying to obtain specific outputs from language models like GPT-3.5. By gradually refining the prompt and giving explicit examples, the user can guide the model towards producing results that align closely with the requirements.


We save the generated criteria in a text file so that it can be used to calculate score in the next assignment

Now we have to download the textfile **criterion_and_string_match_output.txt** so that we can use the string match text and the criteria generated in the next assignments

In [ ]:
from google.colab import files

# List of file paths that you want to download
file_path = "/content/criterion_and_string_match_output.txt"

# Download each file to your local system
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>